In [37]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic

In [ ]:
load_dotenv()  # .env 파일의 환경변수 로드
api_key = os.getenv"(api_key")

NameError: name 'load_dotenv' is not defined

In [ ]:
# ========== 1. 경로 설정 ==========
PDF_PATH = r"C:\Project\web\round12\KB 실버암 간편건강보험Plus.pdf"


In [ ]:
# ========== 2. PDF 로드 ==========
loader = PyPDFLoader(PDF_PATH)
pages = loader.load_and_split()

In [ ]:
# ========== 3. 텍스트 청크 분할 ==========
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)
docs = text_splitter.split_documents(pages)


In [ ]:
# ========== 4. OpenAI 임베딩 ==========
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=api_key # 환경 변수에서 불러오기
)

NameError: name 'api_key' is not defined

In [ ]:
# ========== 5. 벡터 저장소 생성 (FAISS) ==========
vectorstore = FAISS.from_documents(docs, embedding=embeddings)
retriever = vectorstore.as_retriever()


In [ ]:
# ========== 6. Claude 3 모델 설정 ==========
llm = ChatAnthropic(
    model="claude-3-opus-20240229",
    temperature=0,
    max_tokens=1024,
    anthropic_api_key= api_key
)


In [ ]:
# ========== 7. 프롬프트 템플릿 ==========
def rag_prompt(context, question):
    return f"""당신은 보험 약관 분석 전문가입니다.
아래 컨텍스트를 참고해 질문에 답하세요.
문서에 없는 내용은 '문서에서 관련 정보를 찾을 수 없습니다'라고 답변하세요.

컨텍스트:
{context}

질문:
{question}

답변:"""

In [ ]:
# ========== 8. RAG 체인 ==========
def rag_chain(question):
    # 1) 관련 문서 검색
    relevant_docs = retriever.get_relevant_documents(question)
    context_text = "\n\n".join([doc.page_content for doc in relevant_docs])

    # 2) Claude 프롬프트 생성
    prompt_text = rag_prompt(context_text, question)

    # 3) Claude 호출
    response = llm.invoke(prompt_text)
    return response.content

In [ ]:
# ========== 9. 실행 ==========
if __name__ == "__main__":
    # 예시 질문
    question = "평형기능의 장해가 있는거 같아 혹시 기준표나 점수가 있을까?"
    answer = rag_chain(question)
    print(answer)

네, 평형기능의 장해에 대한 평가 기준표와 점수가 문서에 제시되어 있습니다. 

평형기능의 장해 평가 기준은 다음과 같습니다:

"'평형기능에 장해를 남긴 때'라 함은 전정기관 이상으로 보행 등 일상생활이 어려운 상태로 아래의 평형장해 평가항목별 합산점수가 30점 이상인 경우를 말한다.

항목 내용 점수
검사소견 양측 전정기능 소실 14
양측 전정기능 감소 10
일측 전정기능 소실 4"

즉, 양측 전정기능이 소실되면 14점, 양측 전정기능이 감소하면 10점, 일측 전정기능이 소실되면 4점으로 평가하며, 이 점수들의 합산 점수가 30점 이상이면 평형기능에 장해가 있다고 판정합니다.


In [ ]:
# ========== 9. 실행 ==========
if __name__ == "__main__":
    # 예시 질문
    question = """
1. 암보험 가입 후 2개월이 지나서 위암 판정을 받았어 보상받을 수 있나요?
2. 평형기능의 장해가 있는거 같아 혹시 기준표나 점수가 있을까?
3. 제자리암 진단비 지급금액에 대해서 자세히 알려줘
"""
    answer = rag_chain(question)
    print(answer)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************L20A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}